In [107]:
import pandas as pd
import json

In [165]:
total_id = 0
class Mention:
    datetime_happened = None
    datetime_reported = None
    longitude = None
    lattitude = None
    type = None
    # Please use standard system: SVO, SCO, TSE
    airport_id = None
    city_name = None
    country_name = None
    raw_description = None

    
    def __init__(self, type, airport_id, datetime_reported, 
                 datetime_happened=None, lattitude=None, longitude=None, 
                 city_name=None, country_name=None, raw_description=None):
        
        self.datetime_happened = datetime_happened
        self.datetime_reported = datetime_reported
        self.longitude = longitude
        self.lattitude = lattitude
        self.type = type
        self.airport_id = airport_id
        self.city_name = city_name
        self.country_name = country_name
        self.raw_description = None
        
        
class Event(Mention):
    mentions = []
    flights_affected = []
    duration = 0
    id = None
#     def __init__(self, m:Mention):
#         flights_affected = []
#         super().__init__(m.type, m.airport_id, m.datetime_reported, 
#                          m.datetime_happened, m.lattitude, m.longitude, 
#                          m.city_name, m.country_name, None)
        
#         self.mentions.append(m)

    def __init__(self, type, airport_id, datetime_reported, 
                 datetime_happened=None, lattitude=None, longitude=None, 
                 city_name=None, country_name=None, duration=None, raw_description=None):
        
        super().__init__(type, airport_id, datetime_reported, 
                         datetime_happened, lattitude, longitude, 
                         city_name, country_name, raw_description)
        
        self.flights_affected = []
        self.duration = duration
        
        global total_id
        self.id = total_id
        total_id += 1
        
        
    def __eq__(self, other):
        
        if self.duration != self.duration:
            return False
        
        if self.duration == None:
            return (self.airport_id == other.airport_id 
                and self.datetime_happened == other.datetime_happened
                and self.type == self.type)
        else:
            return (self.airport_id == other.airport_id 
                and abs(self.datetime_happened.value - other.datetime_happened.value) < self.duration.value
                and self.type == self.type)
        
    
    def get_confidence():
        pass

In [186]:
flights = pd.read_csv('Flights_schedule.csv', index_col = 'Id')
flights['Departure datatime'] = pd.to_datetime(flights['Departure datatime'])
flights['Arrival datatime'] = pd.to_datetime(flights['Arrival datatime'])
# t = set((flights['Arrival port'].values, flights['Departure port'].values))
# len(t)

f = flights.loc[0]

In [75]:
# flights[flights['Arrival datatime'] - Timestamp.today() < Timedelta('3 days')][flights['Arrival datatime'] - Timestamp.today() > Timedelta('3:00:00')]

In [110]:
from pandas._libs.tslib import Timestamp, Timedelta
import pickle
import numpy as np
e = Event(datetime_happened = Timestamp.today(),
                  datetime_reported = Timestamp.strptime('28/11/2017', '%d/%m/%Y'), 
                  airport_id = 'PEK', type = 'strike', 
                  city_name = 'Helsinki', country_name = 'Finland')

In [112]:
t = Timestamp(1234567890)
type(t.value)

int

In [ ]:
import pyowm
owm = pyowm.OWM('fd7f55cd600a4e84dc1cb6797915f3ee')
airport_db = pd.read_csv("airports.csv")
def air2coords(name):
    lat = airport_db[airport_db['iata'] == name].latitude.values[0]
    long = airport_db[airport_db['iata'] == name].longitude.values[0]
    return float(lat), float(long)

type(air2coords('HEL')[0])

In [5]:
def place2air(city, country):
    return airport_db[airport_db['city'] == city][airport_db['country'] == country]['iata'].values

place2air('Moscow', 'Russia')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


array(['SVO', 'VKO', 'DME', 'BKA', '\\N', '\\N'], dtype=object)

In [166]:
class FlightControlSystem():
    schedule = None
    event_list = []
    port_list = []
    forecasts = {}
    
    def __init__(self, sourse='Flights_schedule.csv'):
        self.event_list = []
        temp = pd.read_csv('Flights_schedule.csv')
        temp['Departure datatime'] = pd.to_datetime(temp['Departure datatime'])
        temp['Arrival datatime'] = pd.to_datetime(temp['Arrival datatime'])
        
        self.port_list = set(temp['Departure port'].values)
        self.port_list.update(set(temp['Arrival port'].values))
        
        self.schedule = self.actualizer(temp)
        
    def actualizer(self, sch, after_date = None):
        if after_date is None:
            after_date = Timestamp.today()
            
        return sch[sch['Arrival datatime'] > after_date]
    
    def add_event(self, event):
        if self.check_similars(event):
            return []
        
        time_area = '1 days' if event.duration == None else event.duration
        
        if event.airport_id == None:
            if event.city_name == None or event.country_name == None:
                print('Incorrect event. Make it ok and try again')
                return []
            
            air_ids = place2air(event.city_name, event.country_name)
        else:
            air_ids = [event.airport_id]
            
        
        ids = []
        print(len(air_ids))
        print(air_ids)
        for air_id in air_ids:
            if air_id not in self.port_list:
                print('mimo')
                continue
                
            by_port_arriv = self.schedule[self.schedule['Arrival port'] == air_id].index
            by_port_dept = self.schedule[self.schedule['Departure port'] == air_id].index

            by_date_dept = self.schedule[self.schedule['Departure datatime'] - event.datetime_happened < time_area][event.datetime_happened  - self.schedule['Departure datatime']< time_area].index
            by_date_arriv = self.schedule[self.schedule['Arrival datatime'] - event.datetime_happened < time_area][event.datetime_happened  - self.schedule['Departure datatime']< time_area].index

            ids = np.concatenate((np.intersect1d(by_port_arriv, by_date_arriv),np.intersect1d(by_port_dept, by_date_dept)))

#             print('by port {}'.format(len(by_port_arriv) + len(by_port_dept)))
#             print('by date {}'.format(len(by_date_arriv) + len(by_date_dept)))
#             print(ids)

            event.flights_affected += ids.tolist()
        self.event_list.append(event)
        
        return ids
    
    def check_similars(self, event):
        if event in self.event_list:
            pass
            #do somethind..................
        else:
            return False
        
    
    def events_by_fid(self, fid):
        return [e for e in self.event_list if fid in e.flights_affected]
    
    def flights_by_event(self, ids):
        l = [e.flights_affected for e in self.event_list is e.id in ids]
        return sum(l, [])
    
    def create_event(self,port, time, type, value):
        e = Event(airport_id=port, 
                          type='snow', 
                          datetime_reported=Timestamp.today(), 
                          datetime_happened=time, 
                          raw_description=value, duration = Timedelta('3:00:00'))
        self.add_event(e)
        return e
        
    
    def data_request(self):
        for i, port in enumerate(self.port_list):
            if i % 50 == 9:
                time.sleep(60)
               
            coords = air2coords(port)
            self.forecasts[port] = owm.three_hours_forecast_at_coords(*coords)
            
        
    def weather_update(self):
        three_day_flights = self.schedule[self.schedule['Arrival datatime'] - Timestamp.today() < Timedelta('3 days')]
        three_day_flights = three_day_flights[three_day_flights['Departure datatime'] - Timestamp.today() > Timedelta('3:00:00')].index
        for i in three_day_flights:
#             print('%2d from %d, event_list: %d' %(i, len(three_day_flights), len(self.event_list)), end='\r')
            flight = self.schedule.loc[i]
            
            for stuff in ['Departure ', 'Arrival ']:

                forecast = self.forecasts[flight[stuff + 'port']]
                weather = forecast.get_weather_at(flight[stuff + 'datatime'])

                snow = weather.get_snow().get('3h', 0)
                if snow > 3: # Attention! Magic numbers!
                    e = self.create_event(flight[stuff + 'port'], flight[stuff + 'datatime'], 'snow', snow)
                    print('new snow event ',snow, ' ', len(e.flights_affected), ' ', e.datetime_happened)

                wind = weather.get_wind().get('speed', 0)
                if wind > 7: # Attention! Magic numbers!
                    e = self.create_event(flight[stuff + 'port'], flight[stuff + 'datatime'], 'strong wind', wind)
                    print('new wind event ',wind, ' ', len(e.flights_affected), ' ', e.datetime_happened)

                # to be continued .......


            
                
    def dump(self):
        with shelve.open('data/events.idx', writeback=True) as db:
            db['last_event_id'] = None
            db['event_list'] = self.event_list
            db['schedule'] = self.schedule
            db['port_list'] = self.port_list
            db['forecast'] = self.forecasts
        
    

In [167]:
e = Event(datetime_happened = Timestamp.today(),
                  datetime_reported = Timestamp.strptime('28/11/2017', '%d/%m/%Y'), 
                  airport_id = 'PEK', type = 'strike', 
                  city_name = 'Moscow', country_name = 'Russia')


e.datetime_happened.value

1511686123016004000

In [168]:
fcs = FlightControlSystem()
fcs.forecasts = pickle.load( open('forecasts.p', 'rb'))
iss = fcs.add_event(e)
len(fcs.port_list)

1
['PEK']


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:51: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:52: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


172

In [169]:
# import pickle
# pickle.dump(fcs.forecasts, open('forecasts.p', 'wb'))

In [170]:
%%capture
fcs.weather_update()

In [171]:
len(fcs.event_list)

48

In [172]:
fcs.dump()

In [201]:
class Encoder(JSONEncoder):
        def default(self, o):
            if isinstance(o, list):
                return o
            elif isinstance(o, Event):
                json = o.__dict__.copy()
                json['datetime_happened'] = o.datetime_happened.value
                json['datetime_reported'] = o.datetime_reported.value
                return json
            else:
                return o.__dict__ 
            
            
            
def flight2json(flight, fid):
    json = flight.to_dict().copy
    print(json)
#     json['Departure datatime'] = flight['Departure datatime'].value
#     json['Arrival datatime'] = flight['Arrival datatime'].value
#     json['Id'] = fid
    
    return json

In [202]:
import shelve, json
def get_all_events():
    with shelve.open('data/events.idx') as db:
        event_list = db['event_list']
        return Encoder().encode(fcs.event_list)
    
    
def get_flights_by_events(ids):
    with shelve.open('data/events.idx') as db:
        event_list = db['event_list']
        schedule = db['schedule']
        
        flight_ids = []
        for event in event_list:
            if event.id in ids:
                flight_ids += event.flights_affected
                
        flight_ids = list(set(flights))
        
        json = [flight2json(schedule.loc[i], i) for i in flight_ids]
        return json
        
        
        
        
        

In [203]:
get_flights_by_events([0, 1])

KeyError: 'the label [Departure datatime] is not in the [index]'